In [1]:
import random
import gc
import argparse
import os
import time
import copy
import pandas as pd
import numpy as np
from collections import defaultdict

# Sklearn
from sklearn.model_selection import StratifiedKFold

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# WandB
import wandb

# Custom Import
from utils.loader import get_loaders
from utils.train_utils import train_one_epoch, valid_one_epoch
from configs import filter_config
from utils.models import build_model, load_model
from utils.metric import get_metrics

/opt/conda/envs/bollworm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
CFG = filter_config.get_config()

In [14]:
train_filter = pd.read_csv("data/preproc/train_filter.csv")
train_filter


,image_id_worm,bw,fold
0,id_0002ea6f15c7fa6f4c221783.jpg,1,2.0
1,id_0005ef295aafe6acc63587db.jpg,1,0.0
2,id_00084298dd030a500033ff78.jpg,0,4.0
3,id_00093f2c76f6488737325859.jpg,1,1.0
4,id_000b2e6c437c643f25d4a6c3.jpg,1,2.0
...,...,...,...
9132,id_ffe6e193dd78b7258d864f6b.jpg,1,4.0
9133,id_ffe98feb1e592e887b81c3f0.jpg,1,4.0
9134,id_ffef90ecf47baa38434f84fb.jpg,1,4.0
9135,id_fff06bb2e59bfce437ac9fc0.jpg,0,3.0


In [15]:
fold=0
val_df = train_filter.query("fold==@fold").reset_index(drop=True)
val_df

,image_id_worm,bw,fold
0,id_0005ef295aafe6acc63587db.jpg,1,0.0
1,id_00750938fb12fad4fd8fad24.jpg,1,0.0
2,id_009d7e2ef3c4f47b14c6c4c3.jpg,0,0.0
3,id_00e5892f475750f80a9f9ce5.jpg,0,0.0
4,id_0128f4371f1b1a678062cff6.jpg,1,0.0
...,...,...,...
1823,id_ff239c464c474c0936fd5d9c.jpg,0,0.0
1824,id_ff52ef03790a72f1ca7c6837.jpg,1,0.0
1825,id_ff6c6dc955223c93456ae3b9.jpg,1,0.0
1826,id_ffc5b54765267f4aa7f015ca.jpg,1,0.0


In [16]:
val_loader= get_loaders(val_df,phase="val")

In [17]:
!ls runs/handle_white_neg/datacorrected_efficientnet_b4_aug++

checkpoint.pth.tar  incorrect_preds.csv  test_incorrect_preds.csv
history.pth	    model_best.pth.tar


In [18]:
run_path = "runs/handle_white_neg/datacorrected_efficientnet_b4_aug++"
chk_path = f"{run_path}/checkpoint.pth.tar"
state_dict  =torch.load(chk_path)["state_dict"]
model =load_model(state_dict).to(CFG.device)

Model Architecture: efficientnet_b4


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Loaded Model Stored


In [19]:

criterion = nn.BCEWithLogitsLoss()

In [20]:
val_loss, val_scores,val_data = valid_one_epoch(model, val_loader, criterion, device=CFG.device, epoch=0,infer=True)

Valid : 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:05<00:00,  5.05it/s, gpu_memory=2.24 GB, valid_loss=0.2029]


In [21]:
len(val_data["preds"])

1828

In [22]:
val_df["preds"] = val_data["preds"].cpu().detach().numpy()
val_df

,image_id_worm,bw,fold,preds
0,id_0005ef295aafe6acc63587db.jpg,1,0.0,1.0
1,id_00750938fb12fad4fd8fad24.jpg,1,0.0,1.0
2,id_009d7e2ef3c4f47b14c6c4c3.jpg,0,0.0,0.0
3,id_00e5892f475750f80a9f9ce5.jpg,0,0.0,0.0
4,id_0128f4371f1b1a678062cff6.jpg,1,0.0,1.0
...,...,...,...,...
1823,id_ff239c464c474c0936fd5d9c.jpg,0,0.0,0.0
1824,id_ff52ef03790a72f1ca7c6837.jpg,1,0.0,1.0
1825,id_ff6c6dc955223c93456ae3b9.jpg,1,0.0,1.0
1826,id_ffc5b54765267f4aa7f015ca.jpg,1,0.0,1.0


In [23]:
val_scores

(0.937636761487965, 0.9552941176470588)

In [24]:
preds =torch.tensor(list(val_df["preds"])).to(CFG.device)
labels = torch.tensor(list(val_df["bw"])).to(CFG.device)
get_metrics(preds, labels)

(0.937636761487965, 0.9552941176470588)

In [14]:
incorrect_preds =val_df[val_df["bw"]!=val_df["preds"]][["image_id_worm","fold","bw","preds"]]
incorrect_preds

,image_id_worm,fold,bw,preds
24,id_03e99321b0ef699d61d998a4.jpg,0.0,0,1.0
69,id_09bf93df720e7999d5f0f8b2.jpg,0.0,1,0.0
92,id_0cdcf54855d2007ceba043f6.jpg,0.0,0,1.0
106,id_0e1dcbe8607e2b9571cfbc0a.jpg,0.0,0,1.0
115,id_0f70fbfe9040967394c18441.jpg,0.0,1,0.0
...,...,...,...,...
1754,id_f4523309bc87b2340a5b4aa1.jpg,0.0,0,1.0
1766,id_f6561725e1d389a73e9e1cdf.jpg,0.0,0,1.0
1771,id_f76f903e93da713c8523f4aa.jpg,0.0,0,1.0
1774,id_f7aa8ec2a24ee73582f04287.jpg,0.0,0,1.0


In [1]:
incorrect_preds.to_csv(f"{run_path}/incorrect_preds.csv",index=False)

NameError: name 'incorrect_preds' is not defined

In [ ]:
incorrect_preds["bw"].value_counts()

## False Positives
1. white color prediction
2. id_0379805694b903d042422c26, other insects
3. id_07c215ce23d27572c799c5d1, wrong label



In [ ]:
incorrect_preds[incorrect_preds["bw"]==0]

## False Negative
1. id_fb8a3fbff03805ece9078afb, dust maybe?
2. id_020985f3f6038e28674b15a9, blur, defocus
3.  very small

In [27]:
incorrect_preds[incorrect_preds["bw"]==1]

,image_id_worm,fold,bw,preds
34,id_050bfb62da75e63107eea8d6.jpg,0.0,1,0.0
51,id_0785d117e88bc09ec93784ec.jpg,0.0,1,0.0
69,id_09bf93df720e7999d5f0f8b2.jpg,0.0,1,0.0
95,id_0d2812b30af07a85453f0c9c.jpg,0.0,1,0.0
115,id_0f70fbfe9040967394c18441.jpg,0.0,1,0.0
127,id_110b5c51dddaeabcc8221d27.jpg,0.0,1,0.0
164,id_16c91018d0e9cb42f908bc82.jpg,0.0,1,0.0
184,id_18dff8f547553da5364e3234.jpg,0.0,1,0.0
192,id_1a80fc01155d090e1048b69b.jpg,0.0,1,0.0
262,id_23a822aa039563134496105e.jpg,0.0,1,0.0


In [40]:
train_filter.bw.value_counts()

1    6179
0    2958
Name: bw, dtype: int64

In [44]:
torch.tensor([1,2,3]).unsqueeze(1).shape

torch.Size([3, 1])

In [45]:
import torch.nn as nn

tensor([0.7311, 0.8808])

In [99]:
y_pred = torch.tensor([12,3,0,0,0.5,1]).unsqueeze(1)
# y_pred = nn.Sigmoid()(y_pred)
y_pred

tensor([[12.0000],
        [ 3.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.5000],
        [ 1.0000]])

In [100]:
labels =  torch.tensor([1,0,0,1,0,1]).unsqueeze(1)
labels

tensor([[1],
        [0],
        [0],
        [1],
        [0],
        [1]])

In [101]:
nn.BCEWithLogitsLoss()(y_pred,labels.float())

tensor(0.9537)

In [79]:
# y_pred = torch.argmax(y_pred, dim=1)

In [80]:
nn.BCEWithLogitsLoss()

tensor([[1.0000],
        [0.9526],
        [1.0000],
        [1.0000],
        [0.6225],
        [0.7311]])

In [109]:
target = torch.zeros([10, 1], dtype=torch.float32)  # 64 classes, batch size = 10
output = torch.full([10, 1], 1.5)  # A prediction (logit)
pos_weight = torch.ones([1])  # All weights are equal to 1
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion(output, target)  # -log(sigmoid(1.5))

tensor(1.7014)

In [112]:
criterion = torch.nn.BCELoss()
criterion(nn.Sigmoid()(output), target)  # -log(sigmoid(1.5))

tensor(1.7014)

In [121]:
y_pred = nn.Sigmoid()(torch.tensor([0,2,3,4]))
y_pred[y_pred>0.6]=1
y_pred[y_pred<0.6]=0

In [122]:
y_pred

tensor([0., 1., 1., 1.])